In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import netCDF4 as nc
import numpy as np

from utility import analysis_parameters as ap

In [2]:
lat0, lat1 = -5, 5
lon0, lon1 = 143, 153
h = "h3" # h3 = omega850,700,500,200; h2 = lw sw fluxes

# get all dates for file var
feb = [ "02-%02d"%(i+1) for i in range(28) ]
dates = ["01-30", "01-31"] + feb
ds = xr.open_dataset(ap.test_data_file_name(h, dates[0])).drop(["P3_input_dim","P3_output_dim","ilev","lev","swband","lwband", "slat","slon"])
ds = ds.sel(lat=slice(lat0, lat1)).sel(lon=slice(lon0,lon1))

for date in dates[1:]:
    file = ap.test_data_file_name(h, date)
    print(file)
    ds_1 = xr.open_dataset(file).drop(["P3_input_dim","P3_output_dim","swband","lwband", "slat","slon"])
    ds_1 = ds_1.sel(lat=slice(lat0, lat1)).sel(lon=slice(lon0,lon1))
    ds = xr.combine_by_coords([ds, ds_1])


/home/disk/eos12/hillmanb/scream/dyamond2/256x512/SCREAMv0.SCREAM-DY2.ne1024pg2.20201127.eam.h3.2020-01-31-00000.nc
/home/disk/eos12/hillmanb/scream/dyamond2/256x512/SCREAMv0.SCREAM-DY2.ne1024pg2.20201127.eam.h3.2020-02-01-00000.nc
/home/disk/eos12/hillmanb/scream/dyamond2/256x512/SCREAMv0.SCREAM-DY2.ne1024pg2.20201127.eam.h3.2020-02-02-00000.nc
/home/disk/eos12/hillmanb/scream/dyamond2/256x512/SCREAMv0.SCREAM-DY2.ne1024pg2.20201127.eam.h3.2020-02-03-00000.nc
/home/disk/eos12/hillmanb/scream/dyamond2/256x512/SCREAMv0.SCREAM-DY2.ne1024pg2.20201127.eam.h3.2020-02-04-00000.nc
/home/disk/eos12/hillmanb/scream/dyamond2/256x512/SCREAMv0.SCREAM-DY2.ne1024pg2.20201127.eam.h3.2020-02-05-00000.nc
/home/disk/eos12/hillmanb/scream/dyamond2/256x512/SCREAMv0.SCREAM-DY2.ne1024pg2.20201127.eam.h3.2020-02-06-00000.nc
/home/disk/eos12/hillmanb/scream/dyamond2/256x512/SCREAMv0.SCREAM-DY2.ne1024pg2.20201127.eam.h3.2020-02-07-00000.nc
/home/disk/eos12/hillmanb/scream/dyamond2/256x512/SCREAMv0.SCREAM-DY2.ne

In [ ]:
if h=="h2":
    olr = ds["FLNT"]
    olr_c = ds["FLNTC"]
    sw_net_toa = ds["FSNTOA"] # net sw flux at toa, FSNTOAC - clear sky net sw flux at toa
    sw_net_toa_c = ds["FSNTOAC"]

    lw_cre = olr_c - olr
    sw_cre = sw_net_toa - sw_net_toa_c
    sw_cre = sw_cre.where(sw_net_toa_c>0)
    net_cre = lw_cre + sw_cre
    
    # plot cre hist
    lw_cre.plot.hist(bins=range(-10,200,10), label="LW (%i)"%(lw_cre.count()), histtype="step")
    sw_cre.plot.hist(bins=range(-400,10,10), label="SW (%i)"%(sw_cre.count()), histtype="step")
    net_cre.plot.hist(bins=range(-200,200,10), label="Net (%i)"%(net_cre.count()), histtype="step")
    plt.legend()
    plt.show()

    # load obs - ceres syn1 1 hourly
    ceres = xr.open_dataset(ap.return_ceres())
    ceres = ceres.isel({"time":slice(29*24,29*24+720)})
    ceres
    
    # plot ceres data
    ceres.adj_atmos_sw_down_clr_toa_1h[0].plot.pcolormesh(vmin=1000,vmax=1240)
    plt.show()
    ceres.adj_atmos_sw_down_all_toa_1h[0].plot.pcolormesh(vmin=1000,vmax=1240)
    plt.show()
    
    # plot cs and all sky net sw toa scream
    sw_net_toa_c[2].plot.pcolormesh(vmin=1000,vmax=1240)
    plt.show()
    sw_net_toa[2].plot.pcolormesh(vmin=700,vmax=1240)
    plt.show()
#     ds.FSDS[2].plot.pcolormesh()
#     plt.show()
    

# Calculate albedo

$ sw_{net} = sw_{up} + sw_{down} $ where down is negative

$ sw_{up} = sw_{net} - sw_{down}  $ 


$ albedo = sw_{up}/sw_{down} = sw_{down}+sw_{net}/sw_{down} = 1 + sw_{net}/sw_{down} = 1 - (sw_{up}/sw_{net}) $

In [ ]:
if h=="h2":
    sw_net_toa_1hrly = sw_net_toa[2::4] # match temporally to ceres 1 hrly
    sw_up_toa = np.zeros(sw_net_toa_1hrly.shape)
    albedo = np.zeros(sw_net_toa_1hrly.shape)
    for i,lat in enumerate(sw_net_toa_1hrly.lat):
        for j,lon in enumerate(sw_net_toa_1hrly.lon):
            down_array = ceres.adj_atmos_sw_down_clr_toa_1h.sel({"lat":lat,"lon":lon}, method="nearest").values
            sw_up_toa[:,i,j] = down_array - sw_net_toa_1hrly[:,i,j].values
            albedo[:,i,j] = np.where(((sw_net_toa_1hrly.time.dt.hour>=0) & (sw_net_toa_1hrly.time.dt.hour<=4)), np.where(down_array>0, sw_up_toa[:,i,j]/down_array, np.nan), np.nan)


    olr = olr.where(olr.time==sw_net_toa_1hrly.time)
    print(olr.shape, albedo.shape)

In [ ]:
if h=="h2":
    plt.pcolormesh(sw_net_toa_1hrly.lon, sw_net_toa_1hrly.lat, sw_up_toa[0,:,:])
    plt.colorbar(label="SW upward at TOA (W/m2)")
    plt.title(str(sw_net_toa_1hrly.time[0].values))
    plt.show()
    plt.pcolormesh(sw_net_toa_1hrly.lon, sw_net_toa_1hrly.lat, albedo[0,:,:])
    plt.colorbar(label="albedo")
    plt.title(str(sw_net_toa_1hrly.time[0].values))
    plt.show()
    print("calculated albedo:\nmin: {}\nmax: {}\nmean: {}\nmedian: {}".format(np.nanmin(albedo), np.nanmax(albedo), np.nanmean(albedo), np.nanmedian(albedo)))

    

In [ ]:
if h=="h2":
    H, X, Y = np.histogram2d(olr.values.flatten(), albedo.flatten(),
                            bins=[np.linspace(80,310,32), np.linspace(0.01,0.8,30)])
    olr_obs = ceres.adj_atmos_lw_up_all_toa_1h
    alb_obs = ceres.adj_atmos_sw_up_all_toa_1h/ceres.adj_atmos_sw_down_all_toa_1h
    olr_obs = olr_obs.where((olr_obs.time.dt.hour>=0)&(olr_obs.time.dt.hour<=4))
    alb_obs = alb_obs.where((alb_obs.time.dt.hour>=0)&(alb_obs.time.dt.hour<=4))
    H_obs, _, _ = np.histogram2d(olr_obs.values.flatten(), alb_obs.values.flatten(),
                            bins=[np.linspace(80,310,32), np.linspace(0.01,0.8,30)])
    print(albedo.shape, olr.shape, olr_obs.shape, alb_obs.shape)

    # plot alb-olr joint histogram
#     plt.pcolormesh(X,Y,np.log10(H.T/(720*14*14)))
#     plt.colorbar(label="log10(pdf)")
#     plt.xlabel("OLR (W/m2)")
#     plt.ylabel("albedo")
#     plt.grid(True)
#     plt.show()

#     plt.pcolormesh(X,Y,np.log10(H_obs.T/(720*10*10)))
#     plt.colorbar(label="log10(pdf)")
#     plt.xlabel("OLR (W/m2)")
#     plt.ylabel("albedo")
#     plt.grid(True)
#     plt.show()

#     plt.pcolormesh(X,Y,(H.T/(720*14*14) - H_obs.T/(720*10*10)))
#     plt.colorbar(label="log10(scream - obs pdf)")
#     plt.xlabel("OLR (W/m2)")
#     plt.ylabel("albedo")
#     plt.grid(True)
#     plt.show()


    # alb-olr jhist as contour plot
    xmid = (X[:-1]+X[1:])/2
    ymid = (Y[:-1]+Y[1:])/2

    plt.figure(figsize=(7,4))
    plt.contourf(xmid,ymid,np.log10(H.T/(720*14*14)), levels=np.arange(-5,-1,0.5))
    plt.colorbar(label="log10(pdf scream)")
    plt.xlabel("OLR (W/m2)")
    plt.ylabel("albedo")
    plt.grid(True)
    plt.contour(xmid,ymid,np.log10(H_obs.T/(720*10*10)),cmap="Greys", levels=np.arange(-5,-1,0.5))
    plt.colorbar(label="log10(pdf ceres syn1deg 1 hrly)")
    plt.show()

    plt.figure(figsize=(5.6,4))
    plt.contourf(xmid,ymid,(H.T/(720*14*14) - H_obs.T/(720*10*10)), cmap="bwr", levels=np.arange(-0.008,0.0081,0.0005))
    plt.colorbar(label="log10(scream - obs pdf)")
    plt.xlabel("OLR (W/m2)")
    plt.ylabel("albedo")
    plt.grid(True)
    plt.show()

# omega

Convert $\omega$ to $w$. 

       rgas = 287.058            ; J/(kg-K) => m2/(s2 K)
       g    = 9.80665            ; m/s2
       rho  = p/(rgas*t)         ; density => kg/m3
       w    = -omega/(rho*g)     ; array operation
       


In [ ]:
# load omega

rgas = 287.058   # J/kg/K
g = 9.80665      # m/s
rho = p/(rgas*t) # kg/m3